# True Range  and Average True Range analysis


In [79]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [80]:
import pandas as pd 
import numpy as np
import time
%memit np.zeros(100000)
%timeit np.zeros(100000)

peak memory: 621.13 MiB, increment: 0.00 MiB
13.4 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [81]:
#load data from book J.Welles Wilder
date = range(1,48)
_open = [52.8,52.6,52,52.2,52.1,51.9,51.5,51.15,51.50,51.6,51,51.35,51.7,51.6,51.55,51.9,52.4,52.2,52,52.5,53.10,53.95,55.20,57.55,57.9,57.75,57.5,57.8,58,58.45,57.8,57,56.3,56.2,56.5,57.25,57.5,57,56.5,56.5,57,57.4,57.2,57.25,58,56.5,56]
high = [53,52.75,52.35,52.45,52.35,52.1,51.8,51.6,51.7,51.6,51.4,51.75,51.9,51.7,51.8,52.5,52.7,52.45,52.65,52.95,53.6,54.5,55.7,57.55,58.55,57.75,58.05,57.9,58.3,58.65,57.8,57.15,56.35,56.6,57.25,57.5,57.5,57.15,56.60,56.9,57.25,57.6,57.25,58,58.5,56.5,56.35]
low = [52.5,52.25,51.85,52.15,51.75,51.5,51,51.25,51.4,51.1,50.75,51.35,51.4,51.15,51.5,51.8,52.1,52,51.5,52.4,53.05,53.8,55,57.55,57.9,57.3,57.15,57.45,57.85,57.75,57,56.25,55.35,56.05,56.4,57,57.25,56.5,56.25,56.45,56.75,57,56.9,57.25,57,55.5,55.75]
close = [52.7,52.55,52.3,52.4,51.9,51.65,51.10,51.55,51.65,51.15,50.75,51.65,51.8,51.55,51.8,52.5,52.1,52.1,52.65,52.9,53.55,54.5,56.55,57.55,58.56,57.65,57.95,57.8,58.2,58.65,57.3,56.6,56.3,56.6,57.25,57.6,57.25,56.6,56.45,56.9,57.1,57.4,57.05,57.9,57,56.05,55.85]
data = {'date': date, 'high': high, 'low':low, 'close': close}

df = pd.DataFrame(data)
print(f"Dataframe shape:{df.shape}")
df

Dataframe shape:(47, 4)


,date,high,low,close
0,1,53.00,52.50,52.70
1,2,52.75,52.25,52.55
2,3,52.35,51.85,52.30
3,4,52.45,52.15,52.40
4,5,52.35,51.75,51.90
5,6,52.10,51.50,51.65
6,7,51.80,51.00,51.10
7,8,51.60,51.25,51.55
8,9,51.70,51.40,51.65
9,10,51.60,51.10,51.15


In [82]:
#load data from csv file
#df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
df = pd.read_csv("data/input/USDT_BTC_300_1425168000_72.csv")
print(f"Dataframe shape:{df.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
df.head()


Dataframe shape:(631047, 11)


,date,high,low,open,close
0,1425168000,240.00006,240.00006,240.00006,240.00006
1,1425168300,240.00006,240.00006,240.00006,240.00006
2,1425168600,240.00006,240.00006,240.00006,240.00006
3,1425168900,240.00006,240.00006,240.00006,240.00006
4,1425169200,240.00006,240.00006,240.00006,240.00006


In [83]:
def true_range_best(df):
    ranges = [df.high - df.low,(df.high - df.close.shift(1)).abs(),(df.low - df.close.shift(1)).abs()]
    return np.maximum.reduce(ranges)
  

In [84]:
def true_range_v1(df):
    ranges = [df.high - df.low,(df.high - df.close.shift(1)).abs(),(df.low - df.close.shift(1)).abs()]
    return pd.DataFrame(ranges).max()

  

In [85]:
def true_range_v3(df): #best for pandas
    df['_tr1'] = df.high - df.low
    df['_tr2']  = (df.high - df.close.shift(1)).abs()
    df['_tr3']  = (df.low - df.close.shift(1)).abs()
    return df[['_tr1','_tr2','_tr3']].max(axis=1)

In [86]:
def true_range_v2(df):
    i = 0
    TR_l = [0]
    while i < df.index[-1]:
        TR = max(df.loc[i + 1, 'high'], df.loc[i, 'close']) - min(df.loc[i + 1, 'low'], df.loc[i, 'close'])
        TR_l.append(TR)
        i = i + 1
    return pd.Series(TR_l)

In [87]:
def average_true_range_v1(tr,n):
    ATR = pd.Series(tr)
    ATR[ATR.index[n-1]] = tr[:n].mean()
    ATR = ATR.drop(ATR.index[:n-1])
    ATR = ATR.ewm(com=n-1,adjust=False).mean()
    return ATR

In [88]:
def average_true_range_v2(tr,n):
    ATR = tr.ewm(com=n-1,adjust=False).mean()
    return ATR

In [89]:
def average_true_range_v3(tr,n): #original from book
    ATR = pd.Series(tr)
    ATR[ATR.index[n-1]] = tr[:n].mean()
    ATR = ATR.drop(ATR.index[:n-1])
    for index,value in ATR.items():
        if index >= n :
            ATR[index] = ((n-1)*ATR[index-1] + ATR[index])/n
    return ATR

In [90]:
def test_true_range(df):

    %timeit df['tr'] = true_range_best(df) 
    #%timeit df['tr_v1'] = true_range_v1(df) #very very slow
    #%timeit df['tr_v2'] = true_range_v2(df) #very very slow
    %timeit df['tr_v3'] = true_range_v3(df) 
    %timeit df['ATR_v1'] = average_true_range_v1(df['tr'],7) 
    %timeit df['ATR_v2'] = average_true_range_v2(df['tr'],7) #faster and finally converge
    #%timeit df['ATR_v3'] = average_true_range_v3(df['tr'],7) #very slow


In [91]:
test_true_range(df)
df.tail()

13.5 ms ± 26.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
49.6 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
23.4 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
4.45 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


,date,high,low,open,close,tr,_tr1,_tr2,_tr3,tr_v3,ATR_v1,ATR_v2
631042,1614555300,45389.721695,45180.000000,45360.849207,45195.925434,209.721695,209.721695,37.858754,171.862942,209.721695,193.979468,193.979468
631043,1614555600,45293.456100,45172.355681,45188.648095,45289.150042,121.100419,121.100419,97.530666,23.569753,121.100419,183.568176,183.568176
631044,1614555900,45264.213936,44900.000000,45264.213936,45027.397996,389.150042,364.213936,24.936105,389.150042,389.150042,212.937014,212.937014
631045,1614556200,45072.922053,44869.676536,45033.742628,44901.864162,203.245516,203.245516,45.524057,157.721460,203.245516,211.552514,211.552514
631046,1614556500,45170.625961,44888.783715,44917.554431,45170.087409,281.842246,281.842246,268.761800,13.080447,281.842246,221.593904,221.593904
